In [1]:
import gensim.downloader

In [2]:
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [3]:
vectors = gensim.downloader.load('word2vec-google-news-300')

In [5]:
vectors.similarity('twitter', 'apple')

0.053352118

In [6]:
import os

In [8]:
os.makedirs('../data/twentyquestions/datasets/word2vec')

In [9]:
import pandas as pd

In [10]:
words = pd.read_csv('../data/twentyquestions/dev-1000.csv')

In [11]:
words

,Words
0,table
1,warhorse
2,gland
3,crossfire
4,anchorage
...,...
995,plank
996,thermograph
997,chloroform
998,participant


In [14]:
word_list = words['Words'].tolist()

In [27]:
test = 'computer'
similarities = []
not_in_index = []
for w in word_list:
    try:
        sim = vectors.similarity(test, w)
    except KeyError:
        not_in_index.append(w)
        similarities.append(0.)
        continue
    similarities.append(sim)
similarities = [(sim - min(similarities)) / (max(similarities) - min(similarities)) for sim in similarities]
tups = list(zip(word_list, similarities))
for i,t in enumerate(tups):
    if t[0] in not_in_index:
        tups[i] = (t[0], 0.)
tups.sort(key=lambda x: -x[1])
word_dataset = [{'Words': t[0], 'Similarity': t[1]} for t in tups]

In [31]:
pd.DataFrame(word_dataset).to_csv(f'../data/twentyquestions/datasets/word2vec/{test}.csv', index=False)

In [42]:
import random
random.seed(42)
test_words = random.choices(list(set(word_list)-set(not_in_index+['computer'])), k=9)
test_words = ['computer'] + test_words
print(test_words)

['computer', 'papa', 'trees', 'child', 'sax', 'crane', 'meatloaf', 'birthstone', 'polyethylene', 'cement']


In [43]:
for test in test_words:
    similarities = []
    not_in_index = []
    # Compute cosine sim
    for w in word_list:
        try:
            sim = vectors.similarity(test, w)
        except KeyError:
            not_in_index.append(w)
            similarities.append(0.)
            continue
        similarities.append(sim)
    # 0-1 normalization
    similarities = [(sim - min(similarities)) / (max(similarities) - min(similarities)) for sim in similarities]
    tups = list(zip(word_list, similarities))
    # Set not_in_index words to 0.
    for i,t in enumerate(tups):
        if t[0] in not_in_index:
            tups[i] = (t[0], 0.)
    # Sort list in descending order of similarity
    tups.sort(key=lambda x: -x[1])
    word_dataset = [{'Words': t[0], 'Similarity': t[1]} for t in tups]
    # Save dataset as a csv
    pd.DataFrame(word_dataset).to_csv(f'../data/twentyquestions/datasets/word2vec/{test}.csv', index=False)